In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import sklearn
import tensorflow

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM ,Dense
from tensorflow.keras.callbacks import TensorBoard

In [3]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [4]:
mp_holistic = mp.solutions.holistic #makes detections
mp_drawing = mp.solutions.drawing_utils #draws detections

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False   #image not writeable
    results = model.process(image)  #make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), #colors landmarks
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) #colors connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    

In [14]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose,face,lh,rh])

In [15]:
""" extract_keypoints(results).shape    #sum of all """

' extract_keypoints(results).shape    #sum of all '

In [16]:
""" pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)    
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
print(pose.shape)   #after flatten (33*4 = 132) flatten is required for lstm model
print(lh.shape) #21*3
print(rh) """

' pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)\nlh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)    \nrh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)\nface = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)\nprint(pose.shape)   #after flatten (33*4 = 132) flatten is required for lstm model\nprint(lh.shape) #21*3\nprint(rh) '

In [17]:
""" len(results.face_landmarks.landmark)*3 """

' len(results.face_landmarks.landmark)*3 '

In [18]:
Data_Path = os.path.join("ISL_Data")

In [20]:
actions = np.array(["good", "hello","thank you"]) #, "your welcome"]),,"morning","night",
no_sequences = 60   #60 videos
sequence_length =30 #30 frames each 

In [34]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(Data_Path, action, str(sequence)))
        except:
            pass

In [35]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        
        for sequence in range(no_sequences):
            
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                              
                draw_landmarks(image, results)
                            
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)               
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(Data_Path, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [24]:
cap.release()
cv2.destroyAllWindows()

In [21]:
label_map = {label:num for num , label in enumerate(actions)}

In [22]:
label_map

{'good': 0, 'hello': 1, 'thank you': 2}

In [23]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(Data_Path, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [24]:
np.array(sequences).shape   #30 videos * 6 classes , 30 frames each , 1662 keypoints in each frame

(180, 30, 1662)

In [25]:
np.array(labels).shape

(180,)

In [26]:
X = np.array(sequences)

In [27]:
X

array([[[ 0.57345933,  0.39202803, -0.83790094, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.56821632,  0.36876953, -0.84878838, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55943131,  0.36008051, -0.83860254, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.52692175,  0.35016912, -0.60718036, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.5280298 ,  0.35015541, -0.62142169, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.52866441,  0.35080031, -0.59430206, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.52989513,  0.35073569, -0.57028186, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.53511584,  0.34848267, -0.65199387, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.53761148,  0.34725961, -0.59628147, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.53743464,  0.34549513, -0.50325799, ...,  

In [28]:
Y = to_categorical(labels).astype(int)  #one hot encoding

In [29]:
Y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1,

In [30]:
X_train , X_test ,Y_train ,Y_test = train_test_split(X, Y , test_size = 0.20)

In [31]:
X_train.shape

(144, 30, 1662)

In [37]:
model = Sequential()
model.add(LSTM( 64 , return_sequences = True, activation = "sigmoid", input_shape= (30,1662)))
model.add(LSTM (128 , return_sequences = True, activation = "sigmoid"))
model.add(LSTM (64 , return_sequences = False, activation = "sigmoid"))
model.add(Dense(64 , activation = "sigmoid"))
model.add(Dense(32 , activation = "sigmoid"))
model.add(Dense(actions.shape[0], activation="softmax"))

In [36]:
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense

# Assuming actions is defined elsewhere in your code and provides the number of output classes
model2 = Sequential()

# Add a Bidirectional LSTM layer
model2.add(Bidirectional(LSTM(64, return_sequences=True, activation="sigmoid"), input_shape=(30, 1662)))

# Add another Bidirectional LSTM layer
model2.add(Bidirectional(LSTM(128, return_sequences=True, activation="sigmoid")))

# Add the final Bidirectional LSTM layer
model2.add(Bidirectional(LSTM(64, return_sequences=False, activation="sigmoid")))

# Add Dense layers
model2.add(Dense(64, activation="sigmoid"))
model2.add(Dense(32, activation="sigmoid"))

# Output layer with softmax activation
model2.add(Dense(actions.shape[0], activation="softmax"))


In [40]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [37]:
model2.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [38]:
model2.fit(X_train, Y_train, epochs=300)

Epoch 1/300
5/5 [==============================] - 4s 83ms/step - loss: 1.1004 - categorical_accuracy: 0.3681
Epoch 2/300
5/5 [==============================] - 0s 90ms/step - loss: 1.1187 - categorical_accuracy: 0.3403
Epoch 3/300
5/5 [==============================] - 0s 91ms/step - loss: 1.1058 - categorical_accuracy: 0.3403
Epoch 4/300
5/5 [==============================] - 0s 90ms/step - loss: 1.1010 - categorical_accuracy: 0.2986
Epoch 5/300
5/5 [==============================] - 0s 92ms/step - loss: 1.0999 - categorical_accuracy: 0.3472
Epoch 6/300
5/5 [==============================] - 0s 90ms/step - loss: 1.0983 - categorical_accuracy: 0.3472
Epoch 7/300
5/5 [==============================] - 0s 88ms/step - loss: 1.0968 - categorical_accuracy: 0.3194
Epoch 8/300
5/5 [==============================] - 0s 88ms/step - loss: 1.0974 - categorical_accuracy: 0.3403
Epoch 9/300
5/5 [==============================] - 0s 89ms/step - loss: 1.0954 - categorical_accuracy: 0.4583
Epoch 10/3

In [39]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 30, 128)          884224    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 30, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                      

In [ ]:
""" model.fit(X_train, Y_train , epochs=2000) """

Epoch 1/2000
6/6 [==============================] - 2s 55ms/step - loss: 1.9225 - categorical_accuracy: 0.1462
Epoch 2/2000
6/6 [==============================] - 0s 53ms/step - loss: 1.8398 - categorical_accuracy: 0.1520
Epoch 3/2000
6/6 [==============================] - 0s 54ms/step - loss: 1.8057 - categorical_accuracy: 0.1754
Epoch 4/2000
6/6 [==============================] - 0s 57ms/step - loss: 1.7970 - categorical_accuracy: 0.1754
Epoch 5/2000
6/6 [==============================] - 0s 54ms/step - loss: 1.7940 - categorical_accuracy: 0.1462
Epoch 6/2000
6/6 [==============================] - 0s 52ms/step - loss: 1.7914 - categorical_accuracy: 0.1754
Epoch 7/2000
6/6 [==============================] - 0s 52ms/step - loss: 1.7927 - categorical_accuracy: 0.1637
Epoch 8/2000
6/6 [==============================] - 0s 49ms/step - loss: 1.7907 - categorical_accuracy: 0.1345
Epoch 9/2000
6/6 [==============================] - 0s 50ms/step - loss: 1.7922 - categorical_accuracy: 0.1637
E

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-tr

In [ ]:
#result = model.predict(X_test)

In [40]:
result = model2.predict(X_test)

In [41]:
for i in range(7):
    print(actions[np.argmax(result[i])])
    print(actions[np.argmax(Y_test[i])])

thank you
thank you
hello
hello
thank you
thank you
hello
hello
hello
hello
good
good
thank you
thank you


In [ ]:
""" model.save("action.h5") """

In [45]:
model2.save("action2.h5")

In [42]:
Y_pred = model2.predict(X_test)

In [43]:
model.evaluate(X_test, Y_test, verbose=1)

NameError: name 'model' is not defined

In [44]:
model2.evaluate(X_test, Y_test, verbose=1)

2/2 [==============================] - 1s 14ms/step - loss: 0.0124 - categorical_accuracy: 1.0000


[0.012386278249323368, 1.0]

In [ ]:
sequence = []
sentence = []
threshold = 0.4



cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret , frame = cap.read()    #return and frame

        image , results = mediapipe_detection(frame, holistic)
        print(results)

        draw_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]

        if len(sentence)== 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
                
        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image , (0, 0) , (640, 40), (235 ,117,16), -1)
        cv2.putText(image, " ".join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255,255), 2, cv2.LINE_AA)


        cv2.imshow("OpenCV Video", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):  #current key == q
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model2.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti